# Análise Exploratória - Composição da Câmara dos Deputados

Neste notebook será efetuada uma análise exploratória utilizando dados abertos disponibilizados pela Câmara dos Deputados.

A Câmara dos Deputados disponibiliza a lista de deputados via API. Veja a documentação da [API](
http://www2.camara.leg.br/transparencia/dados-abertos/dados-abertos-legislativo/webservices) .

In [1]:
#Importação das bibliotecas a serem utilizadas
import requests
import xml.etree.ElementTree as ET
import datetime
import pandas as pd

In [2]:
# Teste de chamada à API para entender o funcionamento
url = 'http://www.camara.gov.br/SitCamaraWS/Deputados.asmx/ObterDeputados'
r = requests.get(url)
raiz = ET.fromstring(r.text)
deputado = raiz[369]
ET.tostring(deputado)

b'<deputado>\n    <ideCadastro>204371</ideCadastro>\n    <codOrcamento />\n    <condicao>Titular</condicao>\n    <matricula>278</matricula>\n    <idParlamentar>1637543</idParlamentar>\n    <nome>FELIPE RIGONI LOPES</nome>\n    <nomeParlamentar>Felipe Rigoni </nomeParlamentar>\n    <urlFoto>http://www.camara.gov.br/internet/deputado/bandep/204371.jpg</urlFoto>\n    <sexo>masculino</sexo>\n    <uf>ES</uf>\n    <partido>PSB</partido>\n    <gabinete>846</gabinete>\n    <anexo>4</anexo>\n    <fone>3215-5846</fone>\n    <email>dep.feliperigoni@camara.leg.br</email>\n    <comissoes>\n      <titular />\n      <suplente />\n    </comissoes>\n  </deputado>\n  '

In [3]:
# Teste de chamada para obter os detalhes
url_detalhes = 'http://www.camara.leg.br/SitCamaraWS/Deputados.asmx/ObterDetalhesDeputado?ideCadastro={}&numLegislatura=56'
url_detalhes = url_detalhes.format('204527')
r = requests.get(url_detalhes)
raiz_ = ET.fromstring(r.text)
detalhe_deputado = raiz_[0]
ET.tostring(detalhe_deputado)

b'<Deputado>\n    <numLegislatura>56</numLegislatura>\n    <email>dep.geninhozuliani@camara.leg.br</email>\n    <nomeProfissao>\n    </nomeProfissao>\n    <dataNascimento>13/01/1976</dataNascimento>\n    <dataFalecimento>\n    </dataFalecimento>\n    <ufRepresentacaoAtual>SP</ufRepresentacaoAtual>\n    <situacaoNaLegislaturaAtual>Em Exerc&#237;cio</situacaoNaLegislaturaAtual>\n    <ideCadastro>204527</ideCadastro>\n    <idParlamentarDeprecated>699056</idParlamentarDeprecated>\n    <nomeParlamentarAtual>Geninho Zuliani </nomeParlamentarAtual>\n    <nomeCivil>EUGENIO JOS&#201; ZULIANI</nomeCivil>\n    <sexo>M</sexo>\n    <partidoAtual>\n      <idPartido>DEM</idPartido>\n      <sigla>DEM</sigla>\n      <nome>Democratas</nome>\n    </partidoAtual>\n    <gabinete>\n      <numero>860</numero>\n      <anexo>4</anexo>\n      <telefone>3215-5860</telefone>\n    </gabinete>\n    <comissoes>\n      <comissao>\n        <idOrgaoLegislativoCD>538786</idOrgaoLegislativoCD>\n        <siglaComissao>PLE

In [4]:
#Dicionário que será utilizado para pegar os dados interessantes
registro = {
    'ideCadastro': deputado.find('ideCadastro').text,
    'condicao': deputado.find('condicao').text,
    'nome': deputado.find('nome').text,
    'nomeParlamentar': deputado.find('nomeParlamentar').text,
    'sexo': deputado.find('sexo').text,
    'dataNascimento': detalhe_deputado.find('dataNascimento').text,
    'profissao':detalhe_deputado.find('nomeProfissao').text,
    'uf': deputado.find('uf').text,
    'partido': deputado.find('partido').text
}

In [5]:
registro

{'ideCadastro': '204371',
 'condicao': 'Titular',
 'nome': 'FELIPE RIGONI LOPES',
 'nomeParlamentar': 'Felipe Rigoni ',
 'sexo': 'masculino',
 'dataNascimento': '13/01/1976',
 'profissao': '\n    ',
 'uf': 'ES',
 'partido': 'PSB'}

In [6]:
#Obtenção dos conjunto total de dados de forma programática
lista = []
i = 0
for deputado in raiz:
    url_detalhes = 'http://www.camara.leg.br/SitCamaraWS/Deputados.asmx/ObterDetalhesDeputado?ideCadastro={}&numLegislatura=56'
    url_detalhes = url_detalhes.format(deputado.find('ideCadastro').text)
    r = requests.get(url_detalhes)
    raiz_ = ET.fromstring(r.text)
    detalhe = raiz_[0]
    registro = {
    'ideCadastro': deputado.find('ideCadastro').text,
    'condicao': deputado.find('condicao').text,
    'nome': deputado.find('nome').text,
    'nomeParlamentar': deputado.find('nomeParlamentar').text,
    'sexo': deputado.find('sexo').text,  
    'uf': deputado.find('uf').text,
    'partido': deputado.find('partido').text,
    'dataNascimento': detalhe.find('dataNascimento').text,
    'email': deputado.find('email').text
}
    lista.append(registro)
df_deputados = pd.DataFrame(lista)

In [7]:
df_deputados.head()

,condicao,dataNascimento,email,ideCadastro,nome,nomeParlamentar,partido,sexo,uf
0,Titular,26/04/1942,dep.beneditadasilva@camara.leg.br,73701,BENEDITA SOUZA DA SILVA SAMPAIO,Benedita da Silva,PT,feminino,RJ
1,Titular,20/12/1953,dep.angelaamin@camara.leg.br,73696,ANGELA REGINA HEINZEN AMIN HELOU,Angela Amin,PP,feminino,SC
2,Titular,20/04/1959,dep.renildocalheiros@camara.leg.br,73801,RENILDO VASCONCELOS CALHEIROS,Renildo Calheiros,PCdoB,masculino,PE
3,Titular,17/05/1957,dep.jandirafeghali@camara.leg.br,74848,JANDIRA FEGHALI,Jandira Feghali,PCdoB,feminino,RJ
4,Titular,07/04/1947,dep.atilalira@camara.leg.br,74459,ÁTILA FREITAS LIRA,Átila Lira,PP,masculino,PI


In [8]:
#Salvar para consultas futuras e evitar ter que baixar novamente
df_deputados.to_csv('deputados.csv', encoding='latin1', sep=';', index='False')

In [9]:
df_deputados.dtypes

condicao           object
dataNascimento     object
email              object
ideCadastro        object
nome               object
nomeParlamentar    object
partido            object
sexo               object
uf                 object
dtype: object

In [10]:
#Para facilitar a análise, criar uma nova coluna com a Idade calculada utilizando a data de nascomento
# Será utilizada uma função que faz o cáculo
def calcular_idade(dt_nascimento):
    hoje = datetime.date.today()
    return hoje.year - dt_nascimento.year - ((hoje.month, hoje.day) < (dt_nascimento.month, dt_nascimento.day))

In [11]:
df_deputados['idade'] = df_deputados.apply(lambda row: calcular_idade(datetime.datetime.strptime(row.dataNascimento, "%d/%m/%Y")), axis=1)

In [12]:
df_deputados.head()

,condicao,dataNascimento,email,ideCadastro,nome,nomeParlamentar,partido,sexo,uf,idade
0,Titular,26/04/1942,dep.beneditadasilva@camara.leg.br,73701,BENEDITA SOUZA DA SILVA SAMPAIO,Benedita da Silva,PT,feminino,RJ,77
1,Titular,20/12/1953,dep.angelaamin@camara.leg.br,73696,ANGELA REGINA HEINZEN AMIN HELOU,Angela Amin,PP,feminino,SC,66
2,Titular,20/04/1959,dep.renildocalheiros@camara.leg.br,73801,RENILDO VASCONCELOS CALHEIROS,Renildo Calheiros,PCdoB,masculino,PE,60
3,Titular,17/05/1957,dep.jandirafeghali@camara.leg.br,74848,JANDIRA FEGHALI,Jandira Feghali,PCdoB,feminino,RJ,62
4,Titular,07/04/1947,dep.atilalira@camara.leg.br,74459,ÁTILA FREITAS LIRA,Átila Lira,PP,masculino,PI,73


Com os dados estabelecidos, podemos tentar algumas consultas para conhecer melhor o conjunto:

In [13]:
#Quais deputados pertencem ao partido 'CIDADANIA'?
df_deputados[df_deputados['partido'] == 'CIDADANIA']

,condicao,dataNascimento,email,ideCadastro,nome,nomeParlamentar,partido,sexo,uf,idade
5,Titular,23/05/1948,dep.rubensbueno@camara.leg.br,73466,RUBENS BUENO,Rubens Bueno,CIDADANIA,masculino,PR,71
53,Titular,08/05/1955,dep.arnaldojardim@camara.leg.br,141391,ARNALDO CALIL PEREIRA JARDIM,Arnaldo Jardim,CIDADANIA,masculino,SP,64
134,Titular,06/10/1962,dep.carmenzanotto@camara.leg.br,164360,CARMEN EMÍLIA BONFÁ ZANOTTO,Carmen Zanotto,CIDADANIA,feminino,SC,57
147,Titular,07/07/1982,dep.marcelocalero@camara.leg.br,204433,MARCELO CALERO FARIA GARCIA,Marcelo Calero,CIDADANIA,masculino,RJ,37
170,Titular,04/11/1978,dep.danielcoelho@camara.leg.br,178916,DANIEL PIRES COELHO,Daniel Coelho,CIDADANIA,masculino,PE,41
199,Titular,22/08/1979,dep.alexmanente@camara.leg.br,178972,ALEX SPINELLI MANENTE,Alex Manente,CIDADANIA,masculino,SP,40
246,Titular,02/06/1971,dep.davitoria@camara.leg.br,204355,JOSIAS MARIO DA VITORIA,Da Vitoria,CIDADANIA,masculino,ES,48
403,Titular,23/06/1973,dep.paulabelmonte@camara.leg.br,204377,PAULA MORENO PARO BELMONTE,Paula Belmonte,CIDADANIA,feminino,DF,46


In [14]:
# Qual a distribuição entre os partidos dos deputados de SP
df_deputados[df_deputados['uf'] == 'SP'].partido.value_counts()

PSDB             9
PSL              9
PT               8
PL               7
REPUBLICANOS     6
DEM              4
PP               4
PSB              3
PODE             3
PSOL             3
NOVO             3
CIDADANIA        2
PSD              2
MDB              2
PDT              1
PCdoB            1
SOLIDARIEDADE    1
PSC              1
PV               1
Name: partido, dtype: int64

In [17]:
#Distribuição dos deputados entre os partidos
df_deputados.partido.value_counts()

PT               53
PSL              53
PP               40
PL               39
PSD              37
MDB              34
PSDB             33
REPUBLICANOS     31
PSB              30
PDT              28
DEM              27
SOLIDARIEDADE    14
PODE             12
PTB              12
PSOL             10
PROS             10
PSC               9
CIDADANIA         8
NOVO              8
PCdoB             8
AVANTE            7
PATRIOTA          5
PV                4
REDE              1
Name: partido, dtype: int64

In [18]:
# Distribuição dos deputados por sexo
df_deputados.sexo.value_counts()

masculino    436
feminino      77
Name: sexo, dtype: int64

Que outras perguntas poderiam ser feitas?? Imagine que você trabalha para uma revista e precisa fazer um infográfico sobre a composição da Câmara na nova legistatura.